# Reserve and configure FABRIC resources for "setting intermediate network"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [3]:
cdn=True

In [4]:
slice_name="intermediate-network-1"
slice = fablib.new_slice(name=slice_name)

This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

In [49]:
import random

sites=[]
total_routers=[]
routers_access_network=random.randint(3,6)
print("routers_access_network: ", routers_access_network)
if cdn:
    number_of_AS=random.randint(2,3)
    print("number_of_AS:", number_of_AS)
    routers_final_network=random.randint(3,4)  
    print("routers_final_network:", routers_final_network)
    for AS in range(number_of_AS):
        if AS==0:
            routers=[1]
            for i in range(routers_access_network-2):
                if i<2:
                    routers.append(1)
                else:
                    routers.append(random.randint(2,3))
            routers.append(1)
        elif AS==number_of_AS-1:
            routers=[1]
            for i in range(routers_final_network-2):
                routers.append(random.randint(2,3))
            routers.append(1)
        else:
            routers=[1]
            hops=random.randint(1,3)
            print("hops in "+str(AS)+" is "+str(hops))
            redundancy=True
            if redundancy:
                for i in range(0,hops-2):
                    routers.append(random.randint(1,3))
                routers.append(1)
            else:
                for i in range(hops-1):
                    routers.append(1)
        total_routers.append(routers)

else:
    number_of_AS=random.randint(3,5)
    print("number_of_AS:" ,number_of_AS)
    routers_final_network=random.randint(3,8)  
    print("routers_final_network:" ,routers_final_network)
    for AS in range(number_of_AS):
        if AS==0:
            routers=[1]
            for i in range(routers_access_network-2):
                if i<2:
                    routers.append(1)
                else:
                    routers.append(random.randint(1,3))
            routers.append(1)
        elif AS==number_of_AS-1:
            routers=[1]
            for i in range(routers_final_network-2):
                routers.append(random.randint(1,3))
            routers.append(1)
        else:
            routers=[1]
            hops=random.randint(1,3)
            print("hops in "+str(AS)+" is"+str(hops))
            redundancy=True
            if redundancy:
                for i in range(0,hops-2):
                    routers.append(random.randint(1,3))
                routers.append(1)
            else:
                for i in range(hops-1):
                    routers.append(1)
        total_routers.append(routers)

print(total_routers)

for r in total_routers:   
    exp_requires = {'core': 2*sum(r)*4, 'smart_nic_1': 2*sum(r)*1}
    while True:
        site_name = fablib.get_random_site()
        if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
            (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['smart_nic_1']) ):
            break

    fablib.show_site(site_name)
    sites.append(site_name)

routers_access_network:  6
number_of_AS: 3
routers_final_network: 4
hops in 1 is 3
[[1, 1, 1, 3, 2, 1], [1, 1, 1], [1, 3, 2, 1]]


Name,MICH
State,Active
Address,"2530 Draper Dr,Ann Arbor, MI 48109"
Location,"(42.2931086, -83.7101319)"
Hosts,3
CPUs,6
Cores Available,366
Cores Capacity,384
Cores Allocated,18
RAM Available,1434
RAM Capacity,1506


Name,SALT
State,Active
Address,"572 Delong Street,Salt Lake City, UT 84104"
Location,"(40.75707505789612, -111.95346637770317)"
Hosts,3
CPUs,6
Cores Available,318
Cores Capacity,384
Cores Allocated,66
RAM Available,1338
RAM Capacity,1506


Name,NCSA
State,Active
Address,"1725 S Oak St.,Champaign, IL 61820"
Location,"(40.1035624, -88.2415105)"
Hosts,3
CPUs,6
Cores Available,342
Cores Capacity,384
Cores Allocated,42
RAM Available,1370
RAM Capacity,1506


In [40]:
file = open("Routers.txt", "w")
file.write(str(total_routers))
file.close()

In [7]:
total_routers=[[1, 1, 1, 3, 2, 1], [1, 1, 1], [1, 3, 2, 1]]

In [8]:
# this cell sets up the routers 
network_names_all = []

for i,router in enumerate(total_routers):
    network_names = []
    for j,r in enumerate(router):
        if r==1:
            network_names.append("as-"+str(i)+"-router-"+str(j)) 
        else:
            for k in range(r): 
                network_names.append("as-"+str(i)+"-router-"+str(j)+chr(k+97))
    network_names_all.append(network_names)

In [9]:
print(network_names_all)

[['as-0-router-0', 'as-0-router-1', 'as-0-router-2', 'as-0-router-3a', 'as-0-router-3b', 'as-0-router-3c', 'as-0-router-4a', 'as-0-router-4b', 'as-0-router-5'], ['as-1-router-0', 'as-1-router-1', 'as-1-router-2'], ['as-2-router-0', 'as-2-router-1a', 'as-2-router-1b', 'as-2-router-1c', 'as-2-router-2a', 'as-2-router-2b', 'as-2-router-3']]


In [ ]:
for i,node in enumerate(network_names_all):
    site_name=sites[i]
    for n in node:
        print(n)
        slice.add_node(name=n, site=site_name, cores=8, ram=128, disk=25, image='default_ubuntu_22')

In [53]:
# this cell sets up the network links 

for x,r in enumerate(total_routers):
    
    nets=[]
    skip1=0
    skip2=routers[0]

    for i in range(len(r)-1):

        j=i+1
        r1=r[i]
        r2=r[j]
        print(i,j)
        print(r1, r2)
        nodes=[]
        for k in range(r1):
            m=skip1+k
            print(m)
            nodes.append(network_names_all[x][m])
        for l in range(r2):
            nodes.append(network_names_all[x][skip2+l])
        nets.append({"name":"as-"+str(x)+"-net"+str(i)+"-"+str(j), "nodes": nodes})
        print("name:as-"+str(x)+"-net"+str(i)+"-"+str(j)+ ",nodes:", nodes)        
        skip1=skip1+r1
        skip2=skip2+r2
    print(nets)

    for n in nets:
        ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
        slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)
                                                                   



0 1
1 1
0
name:as-0-net0-1,nodes: ['as-0-router-0', 'as-0-router-1']
1 2
1 1
1
name:as-0-net1-2,nodes: ['as-0-router-1', 'as-0-router-2']
2 3
1 3
2
name:as-0-net2-3,nodes: ['as-0-router-2', 'as-0-router-3a', 'as-0-router-3b', 'as-0-router-3c']
3 4
3 2
3
4
5
name:as-0-net3-4,nodes: ['as-0-router-3a', 'as-0-router-3b', 'as-0-router-3c', 'as-0-router-4a', 'as-0-router-4b']
4 5
2 1
6
7
name:as-0-net4-5,nodes: ['as-0-router-4a', 'as-0-router-4b', 'as-0-router-5']
[{'name': 'as-0-net0-1', 'nodes': ['as-0-router-0', 'as-0-router-1']}, {'name': 'as-0-net1-2', 'nodes': ['as-0-router-1', 'as-0-router-2']}, {'name': 'as-0-net2-3', 'nodes': ['as-0-router-2', 'as-0-router-3a', 'as-0-router-3b', 'as-0-router-3c']}, {'name': 'as-0-net3-4', 'nodes': ['as-0-router-3a', 'as-0-router-3b', 'as-0-router-3c', 'as-0-router-4a', 'as-0-router-4b']}, {'name': 'as-0-net4-5', 'nodes': ['as-0-router-4a', 'as-0-router-4b', 'as-0-router-5']}]
0 1
1 1
0
name:as-1-net0-1,nodes: ['as-1-router-0', 'as-1-router-1']
1 2
1

When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [ ]:
slice.submit()


Retry: 11, Time: 253 sec


ID,6b1db415-bae2-47ea-8227-fce1fc29d846
Name,intermediate-network-1
Lease Expiration (UTC),2023-07-22 23:35:49 +0000
Lease Start (UTC),2023-07-21 23:35:51 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,Configuring


ID,Name,Site,Type,State,Error
5926c0a3-6d4e-46ce-8d7f-4d8a5d3f111e,as-0-router-0,MICH,node,Active,
2980cd01-71ec-4584-bce8-ab4e24e12e5a,as-0-router-1,MICH,node,Active,
8dd2864b-23a0-4285-840c-95539345e6c6,as-0-router-2,MICH,node,Active,
1724c2cb-2ae5-4867-ae72-8f4fe9252c5f,as-0-router-3a,MICH,node,Active,
1df17483-b371-4130-84f8-ddb91a6c5991,as-0-router-3b,MICH,node,Active,
1eb1d833-559c-42c2-9d86-51a1b41f3154,as-0-router-3c,MICH,node,Active,
bb976c22-6f56-43cf-be61-d07a37915e21,as-0-router-4a,MICH,node,Active,
a4696c7f-1096-4788-8341-79a386400c0a,as-0-router-4b,MICH,node,Active,
907cfef4-fe26-4cd9-98e4-5b538c456ce7,as-0-router-5,MICH,node,Active,
4c3d6dfb-1b7c-42a8-b4a5-173897b373e5,as-1-router-0,SALT,node,Active,


## Configure resources

In [6]:
slice.get_state()

'StableOK'

In [10]:
slice = fablib.get_slice(name='intermediate-network-1')

In [9]:
total_routers=[[1, 1, 1, 3, 2, 1], [1, 1, 1], [1, 3, 2, 1]]

In [7]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

Bring up all of the network interfaces:

In [8]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [16]:
from ipaddress import ip_address, IPv4Address, IPv4Network
for x,routers in enumerate(total_routers):
    skip1=0
    if_name=[]
    for i in range(len(routers)-1):
        ip=1
        j=i+1
        r1=routers[i]
        r2=routers[j]
        print(i,j)
        print(r1, r2)

        for k in range(r1+r2):
            iname=slice.get_interface(network_names_all[x][k+skip1]+"-as-"+str(x)+"-net"+str(i)+"-"+str(j)+"-p1")
            print(iname.get_name())
            if_name.append(slice.get_interface(network_names_all[x][k+skip1]+"-as-"+str(x)+"-net"+str(i)+"-"+str(j)+"-p1"))
            iname.ip_addr_add(addr="10.10."+str(x+1)+str(i)+"."+str(ip), subnet=IPv4Network("10.10."+str(x+1)+str(i)+".0/24"))
            print("addr=10.10."+str(x+1)+str(i)+"."+str(ip)+", subnet=IPv4Network(10.10."+str(x+1)+str(i)+".0/24)")
            print(network_names_all[x][k+skip1]+"-as-"+str(x)+"-net"+str(i)+"-"+str(j)+"-p1") 
            ip=ip+1
        skip1=skip1+r1
        print(if_name)
    

#for iname in if_name:
#    iname.ip_addr_add(addr="10.10."+str(ip)+"."+str(ip), subnet=IPv4Network("10.10."+str(ip+1)+".0/24"))
#   print("addr=10.10."+str(ip)+"."+str(ip), "subnet=IPv4Network(10.10."+str(ip+1)+".0/24)")
#  ip=ip+1


0 1
1 1
as-0-router-0-as-0-net0-1-p1
addr=10.10.10.1, subnet=IPv4Network(10.10.10.0/24)
as-0-router-0-as-0-net0-1-p1
as-0-router-1-as-0-net0-1-p1
addr=10.10.10.2, subnet=IPv4Network(10.10.10.0/24)
as-0-router-1-as-0-net0-1-p1
[<fabrictestbed_extensions.fablib.interface.Interface object at 0x7f5b50273bb0>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f5b501f9460>]
1 2
1 1
as-0-router-1-as-0-net1-2-p1
addr=10.10.11.1, subnet=IPv4Network(10.10.11.0/24)
as-0-router-1-as-0-net1-2-p1
as-0-router-2-as-0-net1-2-p1
addr=10.10.11.2, subnet=IPv4Network(10.10.11.0/24)
as-0-router-2-as-0-net1-2-p1
[<fabrictestbed_extensions.fablib.interface.Interface object at 0x7f5b50273bb0>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f5b501f9460>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f5b501f97c0>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f5b501c14c0>]
2 3
1 3
as-0-router-2-as-0-net2-3-p1
addr=10.10.12.1, subnet=IPv4N

In [11]:
router_nodes=[]
for x,net in enumerate(network_names_all):
    nodes=[slice.get_node(name=n) for n in network_names_all[x]]
    router_nodes.append(nodes)
    

In [12]:
print(len(router_nodes))

3


In [36]:
print(router_nodes)

[[<fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21dd1400>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b219b75b0>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b216d8d30>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b50057eb0>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21d14760>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21f77430>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21920760>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21a7fca0>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21a7f790>], [<fabrictestbed_extensions.fablib.node.Node object at 0x7f5b219b7730>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b219b7a90>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b402dfeb0>], [<fabrictestbed_extensions.fablib.node.Node object at 0x7f5b21f77580>, <fabrictestbed_extensions.fablib.node.Node object at 0x7f5b219c4520>, <fabrictestbed

In [13]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        #print(n.get_name())
        #n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
        #n.execute("sudo apt-get update; sudo apt-get -y install net-tools", quiet=True)
        n.execute("wget -O - https://git.io/JYhs5 | bash")
    

 --2023-07-24 16:06:21--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-24 16:06:23--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.21
Connecting to git.io (git.io)|140.82.113.21|:443... failed: Network is unreachable.
  --2023-07-24 16:06:25--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.21
Connecting to git.io (git.io)|140.82.113.21|:443... failed: Network is unreachable.
  --2023-07-24 16:06:27--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-24 16:06:28--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-24 16:06:30--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.8

In [41]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n)
        n.execute("ifconfig -a")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
ID                 5926c0a3-6d4e-46ce-8d7f-4d8a5d3f111e
Name               as-0-router-0
Cores              8
RAM                128
Disk               100
Image              default_ubuntu_22
Image Type         qcow2
Host               mich-w3.fabric-testbed.net
Site               MICH
Management IP      2607:f018:110:11:f816:3eff:fec4:7caf
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f018:110:11:f816:3eff:fec4:7caf
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
        inet 10.30.6.203  netmask 255.255.254.0  broadcast 10.30.7.255
        inet

In [42]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c'configure terminal\nrouter ospf\nnetwork 10.10.0.0/16 area 0.0.0.0\n exit\n exit\n exit'  ")

 sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vtysh: command not found
  sudo: vtysh: command not found
sudo: vt

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c 'show ip route\nexit'")

In [ ]:
# slice.delete()

In [46]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("wget -O - https://git.io/JYhs5 | bash")

 --2023-07-22 01:36:43--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-22 01:36:45--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-22 01:36:47--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-22 01:36:48--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-22 01:36:49--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
  --2023-07-22 01:36:51--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.21
Connecting to git.io (git.io)|140.8

In [49]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)